### What is CTE and window function in SQL with example

CTE (Common Table Expression) is a temporary, named result set you define at the start of a query; a window function is a function that performs calculations across a set of rows related to the current row without collapsing them into a single row. Together they are often used for cleaner, more powerful analytical SQL.

What is a CTE?
A CTE is like a readable, temporary view that exists only for the duration of one statement and is defined using the WITH keyword before the main SELECT/INSERT/UPDATE/DELETE query.​
It is commonly used to break complex queries into logical steps, reuse the same intermediate result multiple times, and work with hierarchical (recursive) data.​

CTE basic syntax and example
Basic syntax:

```sql
WITH cte_name AS (
    -- intermediate query
    SELECT column1, column2
    FROM some_table
    WHERE condition = true
)
SELECT *
FROM cte_name
WHERE column1 > 100;

```

Example: get employees with above-average salary using a CTE:

```sql
WITH avg_salary AS (
    SELECT AVG(salary) AS avg_sal
    FROM employees
),
high_paid_employees AS (
    SELECT e.employee_id,
           e.employee_name,
           e.salary
    FROM employees e
    CROSS JOIN avg_salary a
    WHERE e.salary > a.avg_sal
)
SELECT *
FROM high_paid_employees;
```

Here avg_salary computes a scalar average once, and high_paid_employees reuses it to filter employees.

### What is a window function?
A window function calculates a value for each row based on a “window” of related rows, defined by OVER (...), but it does not group/aggregate rows away like GROUP BY does.​
Typical window functions include aggregations such as SUM, AVG, COUNT, and ranking/analytic functions like ROW_NUMBER, RANK, DENSE_RANK, LAG, and LEAD.​

Window function syntax and example
General pattern:

```sql
window_function(expression) OVER (
    [PARTITION BY partition_columns]
    [ORDER BY sort_columns]
    [frame_clause]
)
```

Example: rank employees by salary within each department:


```sql
SELECT
    department,
    employee_name,
    salary,
    DENSE_RANK() OVER (
        PARTITION BY department
        ORDER BY salary DESC
    ) AS salary_rank_in_dept
FROM employees;
```
PARTITION BY department restarts the ranking for each department, and ORDER BY salary DESC ranks highest salaries as 1.​

Another example: running total of sales by date:

```sql
SELECT
    sale_date,
    amount,
    SUM(amount) OVER (
        ORDER BY sale_date
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    ) AS running_total
FROM sales;
```

The window here is “all rows from the start up to the current row ordered by date”, so running_total is a cumulative sum.​

Using CTE with window functions together
Combining both lets you structure multi-step analytics cleanly: compute an intermediate set in a CTE, then apply window functions on top.

Example: from employees, first filter to active employees in a CTE, then compute department-wise average salary and rank:

```sql
WITH active_employees AS (
    SELECT *
    FROM employees
    WHERE status = 'ACTIVE'
)
SELECT
    department,
    employee_name,
    salary,
    AVG(salary) OVER (
        PARTITION BY department
    ) AS dept_avg_salary,
    ROW_NUMBER() OVER (
        PARTITION BY department
        ORDER BY salary DESC
    ) AS dept_salary_rank
FROM active_employees;
```

The CTE narrows and organizes the data; the window functions then provide per-row analytics such as per-department average and rank without further grouping.